In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
df = pd.read_csv('data.csv', encoding = 'utf-8')
df.head()
seleccion = ['fecha_fallecimiento',
            'edad', 'sexo', 'criterio_fallecido',
            'dpt_cdc', 'cdc_positividad', 'flag_vacuna',
            'flag_hospitalizado', 'flag_uci', 'con_oxigeno',
            'con_ventilacion', 'evolucion_hosp_ultimo']
df = df[seleccion]
df.head()

,fecha_fallecimiento,edad,sexo,criterio_fallecido,dpt_cdc,cdc_positividad,flag_vacuna,flag_hospitalizado,flag_uci,con_oxigeno,con_ventilacion,evolucion_hosp_ultimo
0,13/04/2021,54.0,M,Criterio SINADEF,LIMA,0,0,0,0,0,0,NaN
1,5/05/2021,27.0,M,Criterio serolÃ³gico,PIURA,1,0,1,0,1,0,defuncion
2,8/04/2021,80.0,F,Criterio SINADEF,LA LIBERTAD,0,0,0,0,0,0,NaN
3,22/07/2021,44.0,F,Criterio serolÃ³gico,LIMA,1,0,0,0,0,0,NaN
4,13/01/2022,85.0,M,Criterio virolÃ³gico,HUANUCO,1,0,0,0,0,0,NaN


In [3]:
def add_LatLong(df):
    mapeo = pd.read_excel('Tabla_Lat_Long.xlsx')
    data = df['dpt_cdc'].copy()
    mapeo.head()

    Lat_Long = dict()
    for i, element  in enumerate(mapeo['Ciudad']):
        Lat_Long[element.lower()] = [mapeo['Latitud'][i], mapeo['Longitud'][i]]
    for i, element in enumerate(data):
        if element.lower() in Lat_Long.keys():
            data[i] = Lat_Long[element.lower()]
        else:
            print(element)
    data = pd.DataFrame((i for i in data), columns = ['lat', 'long'])
    df = pd.concat([df, data], axis = 1)
    return df

In [4]:
df.head()

,fecha_fallecimiento,edad,sexo,criterio_fallecido,dpt_cdc,cdc_positividad,flag_vacuna,flag_hospitalizado,flag_uci,con_oxigeno,con_ventilacion,evolucion_hosp_ultimo
0,13/04/2021,54.0,M,Criterio SINADEF,LIMA,0,0,0,0,0,0,NaN
1,5/05/2021,27.0,M,Criterio serolÃ³gico,PIURA,1,0,1,0,1,0,defuncion
2,8/04/2021,80.0,F,Criterio SINADEF,LA LIBERTAD,0,0,0,0,0,0,NaN
3,22/07/2021,44.0,F,Criterio serolÃ³gico,LIMA,1,0,0,0,0,0,NaN
4,13/01/2022,85.0,M,Criterio virolÃ³gico,HUANUCO,1,0,0,0,0,0,NaN


In [5]:
df.shape[0]

120953

In [6]:
import datetime
df['fecha_fallecimiento'] = pd.to_datetime(df['fecha_fallecimiento'])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120953 entries, 0 to 120952
Data columns (total 12 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   fecha_fallecimiento    120953 non-null  datetime64[ns]
 1   edad                   120952 non-null  float64       
 2   sexo                   120953 non-null  object        
 3   criterio_fallecido     120953 non-null  object        
 4   dpt_cdc                120953 non-null  object        
 5   cdc_positividad        120953 non-null  int64         
 6   flag_vacuna            120953 non-null  int64         
 7   flag_hospitalizado     120953 non-null  int64         
 8   flag_uci               120953 non-null  int64         
 9   con_oxigeno            120953 non-null  int64         
 10  con_ventilacion        120953 non-null  int64         
 11  evolucion_hosp_ultimo  23166 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(6), 

In [8]:
df.head()

,fecha_fallecimiento,edad,sexo,criterio_fallecido,dpt_cdc,cdc_positividad,flag_vacuna,flag_hospitalizado,flag_uci,con_oxigeno,con_ventilacion,evolucion_hosp_ultimo
0,2021-04-13,54.0,M,Criterio SINADEF,LIMA,0,0,0,0,0,0,NaN
1,2021-05-05,27.0,M,Criterio serolÃ³gico,PIURA,1,0,1,0,1,0,defuncion
2,2021-08-04,80.0,F,Criterio SINADEF,LA LIBERTAD,0,0,0,0,0,0,NaN
3,2021-07-22,44.0,F,Criterio serolÃ³gico,LIMA,1,0,0,0,0,0,NaN
4,2022-01-13,85.0,M,Criterio virolÃ³gico,HUANUCO,1,0,0,0,0,0,NaN


In [41]:
data = df.copy()
data_cf = data['criterio_fallecido'].value_counts()
data_cf = pd.DataFrame(data_cf)
data_cf = data_cf.reset_index()

In [42]:
for i, text in enumerate(data_cf['index']):
    print(i, text)

0 Criterio virolÃ³gico
1 Criterio SINADEF
2 Criterio serolÃ³gico
3 Criterio clÃ­nico
4 Criterio investigaciÃ³n EpidemiolÃ³gica
5 Criterio radiolÃ³gico
6 Criterio nexo epidemiolÃ³gico


In [52]:
def change_index_criterio(data_original):
    data = data_original.copy()
    for i, text in enumerate(data['index']):
        if 'virol' in text:
            data.loc[i, 'index'] = 'Virolágico'
        elif 'SINADEF' in text:
            data.loc[i, 'index'] = 'SINADEF'
        elif 'serol' in text:
            data.loc[i, 'index'] = 'Serolágico'
        elif 'investigaci' in text:
            data.loc[i, 'index'] = 'Investigación Epidemiológica'
        elif 'radiol' in text:
            data.loc[i, 'index'] = 'Radiológico'
        elif 'nexo' in text:
            data.loc[i, 'index'] = 'Nexo Epidemiológico'
        else:
            data.loc[i, 'index'] = 'Clínico'
    return data

In [53]:
data_cf = change_index_criterio(data_cf)

In [54]:
data_cf

,index,criterio_fallecido
0,Clínico,61547
1,SINADEF,36771
2,Serolágico,13656
3,Clínico,3254
4,Investigación Epidemiológica,2428
5,Radiológico,2321
6,Nexo Epidemiológico,976


In [74]:
data = df.copy()
data_edad = data['edad'].value_counts()
data_edad = pd.DataFrame(data_edad)
data_edad = data_edad.reset_index()
data_edad.columns = ['Edad', 'Cantidad']

In [75]:
edad = np.array(data_edad['Edad'])
index = np.argsort(edad)
edad = edad[index]
cantidad = np.array(data_edad['Cantidad'])[index]

In [83]:
pd.DataFrame([cantidad], columns = edad)

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,102.0,103.0,104.0,105.0,106.0,107.0,108.0,109.0,110.0,113.0
0,103,85,41,31,30,31,29,29,28,28,...,43,23,12,8,9,1,2,1,2,1


In [84]:
chart_data = pd.DataFrame(
    np.random.randn(20, 3),
    columns=["a", "b", "c"])
chart_data

,a,b,c
0,-2.215693,-0.761490,0.289424
1,0.154753,-0.075647,0.274537
2,0.124037,2.232396,0.417283
3,-0.849280,1.077418,-0.003728
4,-0.698842,2.636726,-0.511375
5,-0.120803,-0.327737,0.293625
6,1.398417,-0.890652,-0.263082
7,-0.460902,1.599007,-0.597300
8,-0.205693,0.979520,-0.248230
9,1.311424,1.883304,-0.158142
